In [1]:
import requests
import pandas as pd
import base64

# Spotify API credentials
client_id = '3366087874b949db86df851fc93b96c8'
client_secret = '99c0035813ae4c648268df946a660839'

# Function to get Spotify access token
def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

    headers = {
        'Authorization': f'Basic {auth_header}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {
        'grant_type': 'client_credentials'
    }

    response = requests.post(auth_url, headers=headers, data=data)
    response.raise_for_status()
    auth_data = response.json()
    return auth_data['access_token']

# Function to search for a track and get its ID
def search_track(track_name, artist_name, token):
    query = f"{track_name} artist:{artist_name}"
    url = f"https://api.spotify.com/v1/search?q={requests.utils.quote(query)}&type=track&limit=1"
    headers = {
        'Authorization': f'Bearer {token}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    json_data = response.json()
    try:
        first_result = json_data['tracks']['items'][0]
        return first_result['id']
    except (KeyError, IndexError):
        return None

# Function to get track details
def get_track_details(track_id, token):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {
        'Authorization': f'Bearer {token}'
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    json_data = response.json()
    image_url = json_data['album']['images'][0]['url']
    return image_url

# Get Access Token
access_token = get_spotify_token(client_id, client_secret)

# Read your DataFrame
df_spotify = pd.read_csv('Most Streamed Spotify Songs 2024.csv', encoding='ISO-8859-1')

# Make sure 'image_url' column exists
if 'image_url' not in df_spotify.columns:
    df_spotify['image_url'] = None

# Loop through each row to get track details and add to DataFrame
for i, row in df_spotify.iterrows():
    print(f"Processing: {row['Track']} by {row['Artist']}")
    try:
        track_id = search_track(row['Track'], row['Artist'], access_token)
        if track_id:
            image_url = get_track_details(track_id, access_token)
            df_spotify.at[i, 'image_url'] = image_url
        else:
            print(f"Track not found: {row['Track']} by {row['Artist']}")
    except Exception as e:
        print(f"Error processing row {i}: {e}")

# Save the updated DataFrame
df_spotify.to_csv('Most Streamed Spotify Songs 2024.csv', index=False)
print("Done. Updated CSV saved.")


Processing: MILLION DOLLAR BABY by Tommy Richman
Processing: Not Like Us by Kendrick Lamar
Processing: i like the way you kiss me by Artemas
Processing: Flowers by Miley Cyrus
Processing: Houdini by Eminem
Processing: Lovin On Me by Jack Harlow
Processing: Beautiful Things by Benson Boone
Processing: Gata Only by FloyyMenor
Processing: Danza Kuduro - Cover by MUSIC LAB JPN
Processing: BAND4BAND (feat. Lil Baby) by Central Cee
Processing: I Had Some Help (feat. Morgan Wallen) by Post Malone
Processing: The Door by Teddy Swims
Processing: LUNCH by Billie Eilish
Processing: Like That by Future
Processing: bathroom floor by Kids With Buns
Processing: LALA by Myke Towers
Processing: Fortnight (feat. Post Malone) by Taylor Swift
Processing: greedy by Tate McRae
Processing: BLUE by Billie Eilish
Processing: As It Was by Harry Styles
Processing: Paint The Town Red by Doja Cat
Processing: Espresso by Sabrina Carpenter
Processing: Danza Kuduro - Cover by MUSIC LAB JPN
Processing: TEXAS HOLD 'EM 